## Tail Calls
*Nov 7, 2022*

### Dynamic Scope
the way in which names are looked up in Scheme and Python is called *lexical scope* (or *static scope*).

**Lexical scope:** The parent of a frame is the environment in which a procedure is ***defined***.

**Dynamic scope:** The parent of a frame is the environment in which a procedure is ***called***.

### Tail Recursion
##### Functional programming
- all functions are pure functions
- no re-assignment and no mutable data types
- name-value bindings are permanent.

Advantages of functional programming:
- the value of an expression is independent of the order in which sub-expressions are evaluated.
- Sub-expressions can safely be evaluated in parallel or on demand (lazily).
- **Referential transparency:** the value of an expression does not change when we substitute one of its sub expression with the value of that sub expression.

In [2]:
# Recursion and Iteration in Python.
def fact(n, k):
    # Time: O(n)
    # Space: O(n)
    if n == 0:
        return k
    else:
        return fact(n - 1, k*n)

def fact_itr(n, k):
    # Time: O(n)
    # Space: O(1)
    while n > 0:
        n, k = n-1, k*n
    return k

The recursive function above in python will have the same run time as the iterative version, but instead has a space of $O(n)$ since it opens a new frame for each iteration.

For scheme, the recursive implementation will have the same Big O space $O(1)$, as the iterative version in python, using tail recursion.

How do we do this? We do this be eliminating the middle man. Eliminating frames that we don't need anymore when we make recursive calls.

In [ ]:
dd

### Tail Calls

A procedure call that has not yet returned is ***active***. Some procedure calls are ***tail calls***. A Scheme interpreter should support an ***unbound number*** of active tail calls using only a ***constant*** amount of space. (skipping over extra frames that are no longer needed.)

A tail call is a call expression in a *tail context:*, what's a tail context?:
- the last body sub-expression in a lambda expression.
  - it's the last one since that last body determines the returned value of the lambda expression.
- Sub-expression 2 & 3 in a tail context `if` expression
- All non-predicate sub-expressions in a tail context `cond`
- the last sub-expression in a tail context `and` or `or`
- The last sub-expression in a tail context `begin`

A call expression is not a tail call if more computation is still required in the calling procedure.

Linear recursive procedures can often be re-written to use tail calls.



##### Example: Length of a list
```py
(define (length s)
    (if (null? s)
        0
        (+ 1 (length (cdr s)))
    )
)
```
Everything is in a tail context except `(length (cdr s))`
after computing the `(length (cdr s))`, there is still more computation required, returning 1 + that.


Lets rewrite it so that it's tail recursive.

```py
(define (length-tail s)
    (define (length-iter s n)
        (if (null? s) n
            (length-itr (cdr s) (+ 1 n))
        )
    )
    (length-itr s 0)
)
```

#### Eval with Tail Call Optimization
The return value of the tail call is the return value of the current procedure call. Therefore, tail calls shouldn't increase the environment size.

##### Example: Reduce
```py
(define (reduce procedure s start)
    (if (null? s) start
        (reduce procedure
                (cdr s)
                (procedure start (car s))))
)
>>> (reduce * (list 3 4 5) 2)
120 # 2 * 3 * 4 * 5
>>> (reduce (lambda (x y) (cons y x)) (list 3 5 5) (list 2))
(5 4 3 2) 
```

Recursive call is a tail call.

Other calls are not; constant space depends on whether `procedure` requires constant space. (we don't know if the procedure is tail recursive.)

##### Example: Map with only a Constant number of Frames

**Not Tail Recursive:**
```py
(define (map procedure s)
    (if (null? s)
        nil
        (cons (procedure (car s))
              (map procedure (cdr s)))
    )
)
```

This is not tail recursive because the sub expressions of cons are not in a tail context.

Tail Recursive Map Definition:


```python
(define (map procedure s)
    (define (map-reverse s m)
        (if (null? s)
            m
            (map-reverse (cdr s)
                         (cons (procedure (car s)) m)
            )
        )
    )
    (reverse (map-reverse s nil))
)

#--------

(define (reverse s)
    (define (reverse-iter s reversed-sofar)
        (if (null? s)
            r
            (reverse-iter (cdr s) (cons (car s) r))
        )
    )
)
```

### General Computing Machines
we create this when we create interpreters. Programs specify the logic of a computational device.